# Пытаемся научить модель генерировать шутки (ну или что-то близкое к шуткам хотя бы)

Мы будем работать с рускоязычной моделью `ruGPT3` от Сбера и библиотекой `transformers` для её дообучения от Hugging Face

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Дообучение модели ruGPt-3
Загружаем модель ruGPT-3 medium. Это русскоязычная модель на основе архитектуры OpenAI, которая была разработана и обучена командами SberDevices, SberAI и SberCloud на открытых данных википедии, художественной литературе, диалогах, программном коде.

In [4]:
model_name_or_path = "sberbank-ai/rugpt3medium_based_on_gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [5]:

tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)

## Как происходит обучение
Обучающий текст нарезается на случайные куски, которые составляются в последовательности из 1024 (2048 у GPT-3) токенов разделяясь специальным `<|endoftext|>` символом. Во время обучения, модель учится предсказывать (классифицировать) каждый токен в последовательности один за другим при помощь `CrossEntropy Loss.`

Так как входная последовательность всегда заполнена до конца, padding не используется. Но во время инференса, длина входного текста может быть произвольной, поэтому надо явно указывать чем паддить оставшиеся позиции. По дефолту использутеся тот же `<|endoftext|>`.

## Обучающие данные
Будем учить GPT генерировать анекдоты.
В библиотеке transformers есть готовые инструменты для подготовки датасета и даталодера. На вход нужен всего лишь один `.txt` файл с обучающим текстом.

In [3]:
# Сохраним обучающие данные в .txt файл 
data_path = 'jokes_dataset.txt'

joke_list = list()

f = open(data_path,'r', encoding='utf-8')
data = f.read()
data_split = data.split('\n\n\n\n')

# Для каждой шутки проставляем тег начала и тег конца
for joke in data_split:
    split_joke = joke.split("\n")
    joke = " ".join(split_joke)
    create_joke = '[SJ] ' + joke + ' [EJ]'
    joke_list.append(create_joke)

joke_list = '\n'.join(joke_list)   

# Получившийся результат запишем в файл tag_jokes.txt
train_path = 'tag_jokes.txt'

with open(train_path, 'w', encoding='utf-8') as t:
    t.write(joke_list)

In [6]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=32)
  
# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


## Training
Для файнтюнинга нам понадобится объект класса Trainer, который сделает всё грязную работу за нас. Далее нужно будет всего-навсего запустить `trainer.train()`

In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='jokes_output',
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=200, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    warmup_steps=10,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16, # to make "virtual" batch size larger
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [8]:
trainer.train()

***** Running training *****
  Num examples = 2066
  Num Epochs = 200
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 6400
  Number of trainable parameters = 355871744
  8%|▊         | 500/6400 [11:22<2:13:11,  1.35s/it]Saving model checkpoint to jokes_output\checkpoint-500
Configuration saved in jokes_output\checkpoint-500\config.json


{'loss': 2.0939, 'learning_rate': 9.233176838810642e-06, 'epoch': 15.62}


Model weights saved in jokes_output\checkpoint-500\pytorch_model.bin
 16%|█▌        | 1000/6400 [22:55<2:03:28,  1.37s/it]Saving model checkpoint to jokes_output\checkpoint-1000
Configuration saved in jokes_output\checkpoint-1000\config.json


{'loss': 1.2778, 'learning_rate': 8.450704225352114e-06, 'epoch': 31.25}


Model weights saved in jokes_output\checkpoint-1000\pytorch_model.bin
 23%|██▎       | 1500/6400 [34:30<1:51:24,  1.36s/it]Saving model checkpoint to jokes_output\checkpoint-1500
Configuration saved in jokes_output\checkpoint-1500\config.json


...

Model weights saved in jokes_output\checkpoint-5500\pytorch_model.bin
 94%|█████████▍| 6000/6400 [2:19:13<09:10,  1.38s/it]Saving model checkpoint to jokes_output\checkpoint-6000
Configuration saved in jokes_output\checkpoint-6000\config.json


{'loss': 0.1902, 'learning_rate': 6.259780907668232e-07, 'epoch': 187.5}


Model weights saved in jokes_output\checkpoint-6000\pytorch_model.bin
100%|██████████| 6400/6400 [2:28:35<00:00,  1.38s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 6400/6400 [2:28:35<00:00,  1.39s/it]

{'train_runtime': 8915.4405, 'train_samples_per_second': 46.347, 'train_steps_per_second': 0.718, 'train_loss': 0.558639839887619, 'epoch': 199.99}


TrainOutput(global_step=6400, training_loss=0.558639839887619, metrics={'train_runtime': 8915.4405, 'train_samples_per_second': 46.347, 'train_steps_per_second': 0.718, 'train_loss': 0.558639839887619, 'epoch': 199.99})

## Сохраняем и загружаем дообученную модель

При обучении сохранили модель в папке `jokes_rugpt3`, потом для удобства при разработке сайта перенесли все файлы модели в папку `saved_model` на гитхабе. Бинарник можно скачать с ГуглДиска: https://drive.google.com/drive/folders/1KFamk4cCb23ZmqMEmFyvhX0Fa1e4FH8n?usp=sharing

In [17]:
output_dir = 'jokes_rugpt3'

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Configuration saved in jokes_rugpt3\config.json
Model weights saved in jokes_rugpt3\pytorch_model.bin
tokenizer config file saved in jokes_rugpt3\tokenizer_config.json
Special tokens file saved in jokes_rugpt3\special_tokens_map.json


('jokes_rugpt3\\tokenizer_config.json',
 'jokes_rugpt3\\special_tokens_map.json',
 'jokes_rugpt3\\vocab.json',
 'jokes_rugpt3\\merges.txt',
 'jokes_rugpt3\\added_tokens.json')

## Результат обучения
Подгружаем нашу дообученную модель из папки `saved_model`

In [1]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT2LMHeadModel.from_pretrained(r"../saved_model").to(DEVICE)

tokenizer = GPT2Tokenizer.from_pretrained(r"../saved_model")

c:\Users\Полина\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ну давайте посмотрим что нам сгенерирует наша модель...

In [7]:
text = "- В буфете негр съел последний багет."
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=70,
                        )

generated_text = list(map(tokenizer.decode, out))[0]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [8]:
extra_list = []
final_joke = ""

for i in range(0, len(generated_text) - 3):
    if ((generated_text[i] == "[") and (generated_text[i + 1] == "E") and
           (generated_text[i + 2] == "J") and (generated_text[i + 3] == "]")):
            break
    if generated_text[i] == "-":
        extra_list.append("\n")
    extra_list.append(generated_text[i])
final_joke += "".join(extra_list) 
print(final_joke)



- В буфете негр съел последний багет. 
- А почему? 
- Не смог остановиться. 


Ещё примерчик

In [26]:
text = "- Как сдать сессию?" 
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=70,
                        )

generated_text = list(map(tokenizer.decode, out))[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [27]:
extra_list = []
final_joke = ""

for i in range(0, len(generated_text) - 3):
    if ((generated_text[i] == "[") and (generated_text[i + 1] == "E") and
           (generated_text[i + 2] == "J") and (generated_text[i + 3] == "]")):
            break
    if generated_text[i] == "-":
        extra_list.append("\n")
    extra_list.append(generated_text[i])
final_joke += "".join(extra_list) 
print(final_joke)


- Как сдать сессию? 
- Очень просто. Раздеться догола и сдать сессию! 
